In [144]:
import pandas as pd
import datetime
import sqlite3

In [145]:
info_comp = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Tabledonnéessupplémentaires.xlsx") # Extraction des info complémentaire, full historique
fafr = pd.read_excel(r"C:\Users\sgasmi\Desktop\Extractions\Effectif_Data_Quality.xlsx") # Requête FaFr

In [146]:
info_comp = info_comp.sort_values(["Matricule", "Date de fin"]).reset_index(drop=True)

In [147]:
info_comp.head(5)

,Matricule,Date de début,Date de fin,Nº de période,Matricule Aramis,Login_SAP
0,3,2018-01-01,2018-12-31,5,A500003,DHEURGUE
1,3,2019-01-01,2019-12-31,5,A500003,dheurgue
2,3,2020-01-01,NaT,5,A500003,dheurgue
3,4,2018-07-23,2018-12-31,6,A500002,CQUEVA
4,4,2019-01-01,2019-12-31,6,A500002,CQUEVA


### Assignation d'une date de fin au 01/01/2050 pour les lignes actives sans date de fin

In [148]:
info_comp.loc[info_comp["Date de fin"].isna(),"Date de fin"] = datetime.datetime(2050, 1, 1)

### Filtre sur les salariés actifs présent dans information complémentaire

In [149]:
info_comp = info_comp.loc[info_comp["Matricule"].isin(fafr["Matricule"])].reset_index(drop=True)

### Remplacement des libellés de colonne par des "_" 

In [150]:
nv_colonnes=[]
for val in info_comp.columns:
    nv_colonnes.append(val.replace(" ", "_"))
    
info_comp.columns = nv_colonnes
print(nv_colonnes)

['Matricule', 'Date_de_début', 'Date_de_fin', 'Nº_de_période', 'Matricule_Aramis', 'Login_SAP']


### Préparation des dates

In [151]:
info_comp["today"] = datetime.datetime.today().date()
info_comp["Date_de_début"] = info_comp["Date_de_début"].dt.date
info_comp["Date_de_fin"] = info_comp["Date_de_fin"].dt.date

In [152]:
# Création d'une base de données in memory
conn = sqlite3.connect(':memory:')

In [153]:
# Ajout de info_comp comme table de la base de données
info_comp.to_sql('info_comp', conn, index=False)

In [154]:
req = "Select Matricule, Date_de_début, Date_de_fin, today from info_comp where today between Date_de_début and Date_de_fin "

In [155]:
# Exécution de la requête et stokage du résultat dans un DataFrame
df = pd.read_sql_query(req, conn)
print(df)

      Matricule Date_de_début Date_de_fin       today
0             3    2020-01-01  2050-01-01  2021-03-03
1             4    2020-01-01  2050-01-01  2021-03-03
2            15    2020-01-01  2050-01-01  2021-03-03
3            18    2020-01-01  2050-01-01  2021-03-03
4            20    2020-01-01  2050-01-01  2021-03-03
...         ...           ...         ...         ...
9751     420705    2021-03-01  2050-01-01  2021-03-03
9752     420706    2021-03-01  2050-01-01  2021-03-03
9753     420715    2021-03-02  2050-01-01  2021-03-03
9754     420716    2021-03-02  2050-01-01  2021-03-03
9755     420717    2021-03-02  2050-01-01  2021-03-03

[9756 rows x 4 columns]


In [156]:
Result = info_comp.loc[info_comp["Matricule"].isin(df["Matricule"])==False]

In [157]:
fafr["Gest"] = fafr.apply(lambda x: str(x["Nom Gest Paie"]) + " " + str(x["Prénom Gest Paie"]), axis=1)

In [158]:
fafr = fafr.loc[fafr["Matricule"].isin(Result["Matricule"])==True]

In [159]:
Result = fafr[["Matricule","Gest"]]

,Matricule,Gest
3949,37296,NACHIT AJBAR OUSILA
4501,38095,NACHIT AJBAR OUSILA
4782,38286,NACHIT AJBAR OUSILA


In [162]:
Result.to_excel(r"C:\Users\sgasmi\Desktop\Export Python\Anomalies info complémentaires.xlsx", index=False)